In [1]:
import os
import numpy as np
import math

data_dir = '/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/'
all_files = os.listdir(data_dir)
all_files = [os.path.join(data_dir, _path) for _path in all_files]
print(all_files)
pred_len = 12
obs_len = 8
threshold = 0.002
min_ped = 1

['/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/students003_train.txt', '/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/biwi_hotel_train.txt', '/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/crowds_zara01_train.txt', '/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/uni_examples_train.txt', '/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/crowds_zara02_train.txt', '/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/students001_train.txt', '/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/crowds_zara03_train.txt']


In [2]:
def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)

In [3]:
def poly_fit(traj, traj_len, threshold):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len) #linspace(start, stop, number)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1] #returns residual for a polynomial fit for x pos
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1] #same for y pos 
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0


In [23]:
import pdb

num_peds_in_seq = []
seq_list = []
seq_list_rel = []
loss_mask_list = []
non_linear_ped = []
for path in all_files:
    print(path)
    data = read_file(path)
    frames = np.unique(data[:, 0]).tolist()
    frame_data = []
    for frame in frames:
        frame_data.append(data[frame == data[:, 0], :])
    print(len(frames), ' frames')
    skip = 1
    seq_len = 20
    num_sequences = int(math.ceil((len(frames) - seq_len + 1) / skip))
    for idx in range(0, num_sequences * skip + 1, skip):
        curr_seq_data = np.concatenate(frame_data[idx:idx + seq_len], axis=0)
        peds_in_curr_seq = np.unique(curr_seq_data[:, 1]) #pedestrian ids per sequence (of frames)
        curr_seq_rel = np.zeros((len(peds_in_curr_seq), 2, seq_len))
        curr_seq = np.zeros((len(peds_in_curr_seq), 2, seq_len))
        curr_loss_mask = np.zeros((len(peds_in_curr_seq),seq_len))
        num_peds_considered = 0
        _non_linear_ped = []
        for _, ped_id in enumerate(peds_in_curr_seq):
            curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] ==
                                         ped_id, :]
            curr_ped_seq = np.around(curr_ped_seq, decimals=4)
            pad_front = frames.index(curr_ped_seq[0, 0]) - idx
            pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
            if pad_end - pad_front != seq_len:
                continue
            curr_ped_seq = np.transpose(curr_ped_seq[:, 2:])
            curr_ped_seq = curr_ped_seq #shape = (2, seq_len) matrix: xpos row, ypos row for each pedestrian id
            # Make coordinates relative
            rel_curr_ped_seq = np.zeros(curr_ped_seq.shape)
            rel_curr_ped_seq[:, 1:] = \
                curr_ped_seq[:, 1:] - curr_ped_seq[:, :-1] #normalized per sequence
            _idx = num_peds_considered
            curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq
            curr_seq_rel[_idx, :, pad_front:pad_end] = rel_curr_ped_seq
            # Linear vs Non-Linear Trajectory
            _non_linear_ped.append(
                poly_fit(curr_ped_seq, pred_len, threshold))
            curr_loss_mask[_idx, pad_front:pad_end] = 1
            num_peds_considered += 1

        if num_peds_considered > min_ped:
            non_linear_ped += _non_linear_ped
            num_peds_in_seq.append(num_peds_considered)
            loss_mask_list.append(curr_loss_mask[:num_peds_considered])
            seq_list.append(curr_seq[:num_peds_considered])
            seq_list_rel.append(curr_seq_rel[:num_peds_considered])

            
print('done')   
num_seq = len(seq_list)
seq_list = np.concatenate(seq_list, axis=0)
seq_list_rel = np.concatenate(seq_list_rel, axis=0)
loss_mask_list = np.concatenate(loss_mask_list, axis=0)
non_linear_ped = np.asarray(non_linear_ped)
print(seq_list.shape)
print(loss_mask_list.shape)
obs_traj = seq_list[:,:,:obs_len]
print(seq_list[1,:,:])
print(sum(loss_mask_list))

/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/students003_train.txt
432  frames
/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/biwi_hotel_train.txt
934  frames
/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/crowds_zara01_train.txt
697  frames
/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/uni_examples_train.txt
587  frames
/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/crowds_zara02_train.txt
841  frames
/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/students001_train.txt
355  frames
/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/train/crowds_zara03_train.txt
603  frames
done
(29809, 2, 20)
(29809, 20)
[[2.273  2.4572 2.6413 2.8255 2.8735 2.8876 2.9017 2.9156 2.9297 2.9438
  2.9577 2.9718 2.9859 2.9998 3.0139 3.028  3.0419 3.056  3.0701 3.0839]
 [6.2052 6.3171 6.4288 6.5407 6.5646 6.5665 6.5684 6.5703 6.5722 6.5739
  6

In [15]:
traj_len = 5
traj = np.random.randn(5)
t = np.linspace(0, traj_len - 1, 5)
res_x = np.polyfit(t, traj, 2, full=True)[1]
print(res_x)

[0.35627098]


In [ ]:
import json

fdir = '/Users/chandrayeebasu/Documents/insight/carladata/data/ma_datum_00000000.json'
with open(fdir) as f:
    d = json.load(f)
for keys in d:
    print(keys)
agent_pasts = np.asarray(d['agent_pasts'])
agent_futures = np.asarray(d['agent_futures'])
player_past = np.asarray(d['player_past'])
player_future = np.asarray(d['player_future'])
print(agent_pasts.shape)
#print(player_past.shape)
#print(agent_futures.shape)
print(agent_pasts[3,:,:])
print(player_past)
print(player_future)